sync linkedin contacts with google contacts?



In [2]:
var importer = require('../Core');

var updateContact, listAllConnections, scrapeEntireLinkedInProfile,
    runSeleniumCell;
var syncLinkedInContacts = () => {
    return importer.interpretAll([
        'update google contact',
        'selenium cell'
    ])
    .then(r => {
        updateContact = r[0].runInNewContext();
        runSeleniumCell = r[1].runInNewContext();
        return runSeleniumCell([
            'scrape linkedin contacts',
            'scrape entire linkedin profile',
        ]);
    })
    .then(r => {
        listAllConnections = r[0];
        scrapeEntireLinkedInProfile = r[1];
        return listAllConnections();
    })
    .then((connections) => importer.runAllPromises(connections.slice(0, 1)
        .map(c => resolve => 
             scrapeEntireLinkedInProfile('https://linkedin.com' + c)
             .then(r => {
                  console.log(r);
                  return updateContact({
                      displayName: r.name.split(/\s/).join('.*')});
             })
             .then(r => resolve(r))
             .catch(e => console.log(e))
            )))
    
};

module.exports = syncLinkedInContacts;
syncLinkedInContacts;



[Function: syncLinkedInContacts]

scrape entire linkedin profile?



In [ ]:
var importer = require('../Core');

var loginLinkedIn, readLinkedInProfileInfo;
var scrapeEntireLinkedInProfile = (profile) => {
    return client
    .then(() => importer.interpretAll([
        'log in linkedin',
        'scrape linkedin profile',
    ]))
    .then(r => {
        loginLinkedIn = r[0].runInNewContext({client, getCredentials});
        readLinkedInProfileInfo = r[1].runInNewContext({client});
        return loginLinkedIn();
    })
    .execute(url => {
        var a = document.createElement('a');
        a.setAttribute('href', url);
        a.href = url;
        a.style.position = 'absolute';
        a.style.zIndex = 4294967295;
        a.style.top = 0;
        a.style.left = 0;
        a.style.bottom = 0;
        a.style.right = 0;
        document.body.appendChild(a);
        setTimeout(() => document.body.removeChild(a), 2000)
    }, profile)
    .click('a[href*="' + profile + '"]')
    .then(() => readLinkedInProfileInfo())
    .catch(e => console.log(e))
}
module.exports = scrapeEntireLinkedInProfile;
scrapeEntireLinkedInProfile;



scrape linkedin contacts?



In [ ]:
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var getAllUntil, getAllXPath;
var listAllConnections = () => {
    var friends = [];
    if(fs.existsSync(project + '/connections.json')) {
        return Promise.resolve(JSON.parse(
            fs.readFileSync(project + '/connections.json')));
    }
    return client
    .then(() => importer.interpretAll([
        'get all elements until',
        'all xpath elements',
    ]))
    .then(r => {
        getAllUntil = r[0].runFresherInNewContext({client});
        getAllXPath = r[1].runFresherInNewContext({client});
        return client.getUrl();
    })
    .url('https://www.linkedin.com/mynetwork/invite-connect/connections/')
    .pause(3000)
    .then(() => getAllUntil(
        false,
        '//a[contains(@href, "/in/")]/@href',
        friends,
        (a, b) => a == b,
        (i) => i < 100
    ))
    .then(r => r.filter((l, i, arr) => arr.indexOf(l) === i))
    .then(r => {
        fs.writeFileSync(
            project + '/connections.json',
            JSON.stringify(r, null, 4))
        return r;
    })
    .catch(e => console.log(e))
};
module.exports = listAllConnections;
listAllConnections;



In [ ]:
var importer = require('../Core');

$$.async();
var syncLinkedInContacts;
importer.interpretAll('sync linkedin contacts')
.then(r => syncLinkedInContacts = r.runInNewContext())
.then(() => syncLinkedInContacts())
.then(r => $$.sendResult(r.map(c => c.names)))
.catch(e => $$.sendError(e))



sync facebook contacts with google contacts?



update google contact?



In [ ]:
var google = require('googleapis');
var people = google.people('v1');
var importer = require('../Core');

var options = {scopes: [
    'https://www.googleapis.com/auth/contacts'
]};

var findMatch = (arr, contact) => {
    return arr.filter(c => typeof c.names !== 'undefined'
                      && c.names[0].displayName
                          .match(new RegExp(contact.displayName, 'ig')));
}

var googlePromise = (func) => {
    return new Promise((resolve, reject) => func((err, response) => {
        if(err) reject(err);
        try {
        } catch (e) {
            reject(e);
        }
        setTimeout(() => resolve(response), 100);
    })).catch(e => console.log(e));
};

var contactCache = [];
var listContacts = (contact, done = false) => {
    // return matching contacts or empty if there are no more pages
    return googlePromise(
        people.people.connections.list.bind(people.people.connections, {
            resourceName: 'people/me',
            personFields: 'emailAddresses,names,birthdays,phoneNumbers',
            auth: options.auth,
            pageToken: options.pageToken,
        }))
    .then(r => {
        options.pageToken = r.nextPageToken;
        done = !options.pageToken; 
        return r.connections;
    })
    .then(r => {
        for(var c of r) {
            contactCache.push(c);
        }
        if(done) {
            const contacts = findMatch(contactCache, contact);
            return contacts;
        }
        return listContacts(contact, done);
    })
    .catch(e => console.log(e))
};

var updateContact = (contact) => {
    var getOauthClient;
    return importer.interpretAll([
        'import google calendar api',
    ])
    .then(r => {
        getOauthClient = r[0].runInNewContext();
    })
    .then(() => getOauthClient(options))
    .then(() => {
        options.nextPageToken = null;
        contactCache = [];
        return listContacts(contact);
    })
    .catch(e => console.log(e))
};

module.exports = updateContact;
updateContact;

